In [8]:
import numpy as np
import time

In [2]:
data = np.load("scratch_cnn_model.npz")

conv1_w = data["conv1_w"]
conv1_b = data["conv1_b"]
conv2_w = data["conv2_w"]
conv2_b = data["conv2_b"]
conv3_w = data["conv3_w"]
conv3_b = data["conv3_b"]
dense1_w = data["dense1_w"]
dense1_b = data["dense1_b"]
dense2_w = data["dense2_w"]
dense2_b = data["dense2_b"]
output_w = data["output_w"]
output_b = data["output_b"]


In [3]:
# Activation
def relu(x): return np.maximum(0, x)
def softmax(x):
    e = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e / np.sum(e, axis=1, keepdims=True)


def flatten(x): return x.reshape(x.shape[0], -1)

# Conv and pooling
def conv2d(x, w, b, stride=1):
    N, H, W, C = x.shape
    F, kH, kW, _ = w.shape
    out_h = (H - kH) // stride + 1
    out_w = (W - kW) // stride + 1
    out = np.zeros((N, out_h, out_w, F))
    for n in range(N):
        for f in range(F):
            for i in range(out_h):
                for j in range(out_w):
                    h_start = i * stride
                    w_start = j * stride
                    region = x[n, h_start:h_start+kH, w_start:w_start+kW, :]
                    out[n, i, j, f] = np.sum(region * w[f]) + b[f]
    return out
def maxpool2d(x, size=2, stride=2):
    N, H, W, C = x.shape
    H_out = (H - size) // stride + 1
    W_out = (W - size) // stride + 1
    out = np.zeros((N, H_out, W_out, C))
    for n in range(N):
        for c in range(C):
            for i in range(H_out):
                for j in range(W_out):
                    h_start = i * stride
                    w_start = j * stride
                    h_end = h_start + size
                    w_end = w_start + size
                    out[n, i, j, c] = np.max(x[n, h_start:h_end, w_start:w_end, c])
    return out

# Evaluate
def evaluate(xd, yd):
    x = conv2d(xd, conv1_w, conv1_b)
    x = relu(x)
    x = maxpool2d(x)
    x = conv2d(x, conv2_w, conv2_b)
    x = relu(x)
    x = maxpool2d(x)
    x = conv2d(x, conv3_w, conv3_b)
    x = relu(x)
    x = maxpool2d(x)
    x = flatten(x)
    x = relu(np.dot(x, dense1_w.T) + dense1_b)
    x = relu(np.dot(x, dense2_w.T) + dense2_b)
    logits = np.dot(x, output_w.T) + output_b
    probs = softmax(logits)
    pred = np.argmax(probs, axis=1)
    label = np.argmax(yd, axis=1)
    return np.mean(pred == label)




In [10]:
x_test = np.load('x_test_small.npy')
y_test = np.load('y_test_small.npy')

In [11]:
print(x_test.shape)
print(y_test.shape)

(100, 32, 32, 3)
(100, 10)


In [13]:
start = time.time()
x_test_eva = evaluate(x_test[0:2], y_test[0:2])
end = time.time()

print(x_test_eva)
print(end-start)

0.0
13.577001571655273
